# Decoupling Simulation Accuracy from Mesh Quality

Simple python notebook to regenrate the teaser image.

The data and scripts to regenerate the other figures can be found 
[here](https://github.com/polyfem/Decoupling-Simulation-Accuracy-from-Mesh-Quality).

Note, we used [polyfem](https://polyfem.github.io) [C++ version](https://github.com/polyfem/polyfem/)  for all figures, with direct solver Pardiso, this notebook uses the default iterative algebraic multigrid Hypre.

Note that this can be run direcly with binder!
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/polyfem/Decoupling-Simulation-Accuracy-from-Mesh-Quality.git/master?filepath=Decoupling-Simulation-Accuracy-from-Mesh-Quality.ipynb)

Importing libraries

In [ ]:
import numpy as np
import polyfempy as pf
import json

#plotting:
import meshplot as mp
import plotly.offline as plotly
import plotly.graph_objs as go
#Necessary for the notebook
plotly.init_notebook_mode(connected=True)

Note that this notebook is designed to be run in the root folder.
If you want to reproduce the results of the teaser you can find the images [here](https://github.com/polyfem/Decoupling-Simulation-Accuracy-from-Mesh-Quality/tree/master/fig1-teaser/meshes).

Setup problem, the exact definition of the function can be found [here](https://github.com/polyfem/polyfem/blob/d598e1764512da28d62ecbbbd1a832407ffb42de/src/problem/MiscProblem.cpp#L23)

In [ ]:
settings = pf.Settings()

#Teaser uses laplacian
settings.set_pde(pf.PDEs.Laplacian)
problem = pf.Problem()
#Use quartic function f = (2*y-0.9)^4+0.1 for bounary conditions
#f''=\Deta f = 48*(2*y-.9)^2 for rhs
problem.add_dirichlet_value(1, "(2*y-0.9)^4+0.1")
problem.exact = "(2*y-0.9)^4+0.1"
problem.exact_grad = ["0", "8*(2*y-.9)^3"]
problem.rhs = "48*(2*y-.9)^2"


settings.problem = problem

First we solve using standard $P_1$ elements and our method which we enable with `settings.set_advanced_option("use_p_ref", True)`.

**Note** this will take some time, the last mesh quality is really low!

In [ ]:
n_meshes = 8

total_solutions = {}
total_errors = {}

for use_pref in [False, True]:
    settings.set_advanced_option("use_p_ref", use_pref)
    
    solver = pf.Solver()
    solver.set_log_level(6)
    solver.settings(settings)

    solutions = []
    errors = []

    #iterating over input files, named from 0 to 8
    for i in range(n_meshes+1):
        solver.load_mesh_from_path("fig1-teaser/meshes/large_angle_strip_{}.obj".format(i), normalize_mesh=True)
        
        #we dont distinguis bc contitions, all sidesets to 1
        solver.set_boundary_side_set_from_bary(lambda v: 1)
        
        solver.solve()
        solver.compute_errors()

        #getting and storing solution
        v, f, sol = solver.get_sampled_solution()
        pts = np.append(v, sol, 1)
        solutions.append({"pts": pts, "f": f})

        #getting error:
        log = solver.get_log()
        log = json.loads(log)
        errors.append(log['err_l2'])
        
    #store these list in the total one
    total_solutions["ours" if use_pref else "p1"] = solutions
    total_errors["ours" if use_pref else "p1"] = errors

print("Done!")

**Plz wait until done!**

## Plot 3D results

For nice plot we append the 2 meshses, ours on the left

In [ ]:
def get_v_f_c(sols, i):
    #append points and offset x by 1.1
    pts_ours = np.array(total_solutions["ours"][i]["pts"])
    pts_p1 = np.array(total_solutions["p1"][i]["pts"])
    pts_p1[:, 0] += 1.1
    v = np.append(pts_ours, pts_p1, 0)
    
    #colors
    c = np.array(v[:, 2])
    
    #append faces
    f_ours = np.array(total_solutions["ours"][i]['f'])
    f_p1 = np.array(total_solutions["p1"][i]['f'])
    f_p1 += np.max(f_ours)+1
    f = np.append(f_ours, f_p1, 0)
    
    return v, f, c

Our results are on the left!

You can rotate the view with the mouse!

In [ ]:
v, f, c = get_v_f_c(total_solutions, 0)
p = mp.plot(v, f, c, return_plot=True)

@mp.interact(mesh=[('mesh_{}'.format(i), i) for i in range(n_meshes+1)])
def ff(mesh):
    v, f, c = get_v_f_c(total_solutions, mesh)
    mp.plot(v, f, c, plot=p)
p

## Error plor

In [ ]:
colors = {'ours': 'rgb(85, 239, 196)', 'p1': 'rgb(255, 201, 26)'}
marker_shape = 'circle'
marker_size = 6

def get_plot_trace(errors, method):
    x = []
    y = []
    
    for i in range(len(errors[method])):
        x.append(i)
        y.append(errors[method][i])
        
    x, y = zip(*sorted(zip(x, y))[:])
    trace = go.Scatter(x=x, y=y,
        mode='lines+markers',
        name=method,
        line=dict(color=colors[method]),
        marker=dict(symbol=marker_shape, size=marker_size)
    )
    return trace

In [ ]:
plot_data = [get_plot_trace(total_errors, "ours"), get_plot_trace(total_errors, "p1")]

layout = go.Layout(
    legend=dict(x=0.1, y=0.98),
    xaxis=dict(
        title="Mesh",
        nticks=5,
        ),
    yaxis=dict(
        title="Error L2",
        range=[0.0, 0.3]
    ),
    hovermode='closest')

fig = go.Figure(data=plot_data, layout=layout)
plotly.iplot(fig)